#Finding best model and hyper parameter tunning using GridSearchCV and RandomizedSearchCV

---



**By using iris flower dataset, we will be finding best model by using best hyper parameters by GridSearchCV**

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [ ]:
df['flower']=iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
flower_mapping = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
df['flower'] = df['flower'].replace(flower_mapping)
df[48:120]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
...,...,...,...,...,...
115,6.4,3.2,5.3,2.3,virginica
116,6.5,3.0,5.5,1.8,virginica
117,7.7,3.8,6.7,2.2,virginica
118,7.7,2.6,6.9,2.3,virginica


**Approach 1: Use train_test_split and manually tune parameters by trial and error**
---



In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(iris.data,iris.target,test_size=0.2)

In [ ]:
from sklearn import svm
model = svm.SVC(kernel='rbf',C=30,gamma='auto')  #{'linear', 'rbf', 'sigmoid', 'poly', 'precomputed'}
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.9

In [ ]:
model = svm.SVC(kernel='linear',C=30,gamma='auto')  #{'linear', 'rbf', 'sigmoid', 'poly', 'precomputed'}
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.9

In [ ]:
model = svm.SVC(kernel='sigmoid',C=30,gamma='auto')  #{'linear', 'rbf', 'sigmoid', 'poly', 'precomputed'}
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.23333333333333334

In [ ]:
model = svm.SVC(kernel='poly',C=30,gamma='auto')  #{'linear', 'rbf', 'sigmoid', 'poly', 'precomputed'}
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.9

In [ ]:
model = svm.SVC(kernel='precomputed',C=30,gamma='auto')  #{'linear', 'rbf', 'sigmoid', 'poly', 'precomputed'}
model.fit(x_train,y_train)
model.score(x_test,y_test)

ValueError: ignored

**Approach 2: Use K Fold Cross validation**
---



In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(svm.SVC(kernel='linear',C=20,gamma='auto'),iris.data,iris.target,cv=6)

array([1.  , 1.  , 0.92, 0.92, 0.96, 1.  ])

In [ ]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data,iris.target,cv=6)

array([0.96, 1.  , 0.92, 0.92, 0.96, 1.  ])

In [ ]:
cross_val_score(svm.SVC(kernel='linear',C=5,gamma='auto'),iris.data,iris.target,cv=6)

array([1.  , 1.  , 0.92, 0.92, 1.  , 1.  ])

**Instead of doing manually, We can use for loop as an alternative**

In [ ]:
import numpy as np
kernels=['rbf','linear','poly']
C=[1,5,10,20]
avg_scores={}
for k in kernels:
  for c in C:
    cv_scores = cross_val_score(svm.SVC(kernel=k,C=c,gamma='auto'),iris.data,iris.target,cv=6)
    avg_scores[k+'_'+str(c)]=np.average(cv_scores)
avg_scores

{'rbf_1': 0.9666666666666667,
 'rbf_5': 0.9666666666666667,
 'rbf_10': 0.96,
 'rbf_20': 0.9533333333333333,
 'linear_1': 0.98,
 'linear_5': 0.9733333333333333,
 'linear_10': 0.9666666666666667,
 'linear_20': 0.9666666666666667,
 'poly_1': 0.9733333333333333,
 'poly_5': 0.9733333333333333,
 'poly_10': 0.9733333333333333,
 'poly_20': 0.96}

**From above results we can say that linear with C=1 will give best performance i.e 0.98**

**Approach 3: Use GridSearchCV**
---

**GridSearchCV do the exactly same thing as for loop above but in a single line of code**

In [ ]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'),
                   {
                      'kernel':['rbf','linear','poly'],
                      'C':[1,5,10,20]
                   },cv=6)
clf.fit(iris.data,iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00130208, 0.00088374, 0.00281568, 0.00092113, 0.00076981,
        0.00354727, 0.00090667, 0.00080836, 0.00503341, 0.00090834,
        0.00079195, 0.00723187]),
 'std_fit_time': array([3.67430199e-04, 8.32238709e-05, 2.25415240e-03, 7.09224478e-05,
        1.28572375e-05, 2.99901998e-03, 2.99995308e-05, 5.26496299e-05,
        3.30964214e-03, 4.52064546e-05, 1.84322217e-05, 3.62962734e-03]),
 'mean_score_time': array([0.0007726 , 0.00057542, 0.0006063 , 0.00061806, 0.00053787,
        0.00061409, 0.00057622, 0.00055631, 0.00077403, 0.00056934,
        0.00055599, 0.00075495]),
 'std_score_time': array([1.80499228e-04, 1.17163185e-05, 1.03791798e-04, 5.75374174e-05,
        1.98556542e-05, 6.42542371e-05, 3.02805414e-05, 3.87769738e-05,
        1.76070291e-04, 1.65765222e-05, 5.16284960e-05, 6.89771431e-05]),
 'param_C': masked_array(data=[1, 1, 1, 5, 5, 5, 10, 10, 10, 20, 20, 20],
              mask=[False, False, False, False, False, False, False, False,
    

In [ ]:
df1 = pd.DataFrame(clf.cv_results_)
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001302,0.000367,0.000773,0.000180,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.96,1.0,0.96,0.92,0.96,1.0,0.966667,0.027487,6
1,0.000884,0.000083,0.000575,0.000012,1,linear,"{'C': 1, 'kernel': 'linear'}",0.96,1.0,0.96,0.96,1.00,1.0,0.980000,0.020000,1
2,0.002816,0.002254,0.000606,0.000104,1,poly,"{'C': 1, 'kernel': 'poly'}",1.00,1.0,0.96,0.92,0.96,1.0,0.973333,0.029814,2
3,0.000921,0.000071,0.000618,0.000058,5,rbf,"{'C': 5, 'kernel': 'rbf'}",0.96,1.0,0.96,0.92,0.96,1.0,0.966667,0.027487,6
4,0.000770,0.000013,0.000538,0.000020,5,linear,"{'C': 5, 'kernel': 'linear'}",1.00,1.0,0.92,0.92,1.00,1.0,0.973333,0.037712,2
5,0.003547,0.002999,0.000614,0.000064,5,poly,"{'C': 5, 'kernel': 'poly'}",1.00,1.0,0.96,0.96,0.92,1.0,0.973333,0.029814,2
6,0.000907,0.000030,0.000576,0.000030,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.96,1.0,0.92,0.92,0.96,1.0,0.960000,0.032660,10
7,0.000808,0.000053,0.000556,0.000039,10,linear,"{'C': 10, 'kernel': 'linear'}",1.00,1.0,0.92,0.92,0.96,1.0,0.966667,0.035901,6
8,0.005033,0.003310,0.000774,0.000176,10,poly,"{'C': 10, 'kernel': 'poly'}",1.00,1.0,0.96,0.96,0.92,1.0,0.973333,0.029814,2
9,0.000908,0.000045,0.000569,0.000017,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.96,1.0,0.88,0.92,0.96,1.0,0.953333,0.042687,12


In [ ]:
df1[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.966667
1,1,linear,0.980000
2,1,poly,0.973333
3,5,rbf,0.966667
4,5,linear,0.973333
5,5,poly,0.973333
6,10,rbf,0.960000
7,10,linear,0.966667
8,10,poly,0.973333
9,20,rbf,0.953333


In [ ]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_fe

In [ ]:
clf.score

<bound method BaseSearchCV.score of GridSearchCV(cv=6, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 5, 10, 20],
                         'kernel': ['rbf', 'linear', 'poly']})>

In [ ]:
clf.best_score_

0.98

In [ ]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

**Use RandomizedSearchCV to reduce number of iterations and with random combination of parameters. This is useful when you have too many parameters to try and your training time is longer. It helps reduce the cost of computation**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    },
    cv=5,
    n_iter=3
)
rs.fit(iris.data, iris.target)
rs.cv_results_

{'mean_fit_time': array([0.00622673, 0.00315828, 0.00594311]),
 'std_fit_time': array([0.00515705, 0.00301768, 0.00567056]),
 'mean_score_time': array([0.00447803, 0.00445137, 0.0009975 ]),
 'std_score_time': array([0.00278882, 0.00433545, 0.00010919]),
 'param_kernel': masked_array(data=['rbf', 'rbf', 'linear'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[10, 20, 1],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'rbf', 'C': 10},
  {'kernel': 'rbf', 'C': 20},
  {'kernel': 'linear', 'C': 1}],
 'split0_test_score': array([0.96666667, 0.96666667, 0.96666667]),
 'split1_test_score': array([1., 1., 1.]),
 'split2_test_score': array([0.96666667, 0.9       , 0.96666667]),
 'split3_test_score': array([0.96666667, 0.96666667, 0.96666667]),
 'split4_test_score': array([1., 1., 1.]),
 'mean_test_score': array([0.98      , 0.96666667, 0.98      ]),

In [ ]:
df2=pd.DataFrame(rs.cv_results_)
df2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006227,0.005157,0.004478,0.002789,rbf,10,"{'kernel': 'rbf', 'C': 10}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.003158,0.003018,0.004451,0.004335,rbf,20,"{'kernel': 'rbf', 'C': 20}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,3
2,0.005943,0.005671,0.000997,0.000109,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1


In [ ]:
df2[['param_kernel','param_C','mean_test_score']]

,param_kernel,param_C,mean_test_score
0,rbf,10,0.980000
1,rbf,20,0.966667
2,linear,1,0.980000


**How about different models with different hyperparameters?**

In [95]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB

In [104]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(n_estimators=40),
        'params' : {
            'n_estimators': [1,10,20]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,10,20]
        }
    },
    'decision_tree':{
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 5, 10, 15, 20],
            'min_samples_split': [2, 5, 10],
        }
    },
    'gaussian_naive_bayes':{
        'model':GaussianNB(),
        'params':{
            'priors': [None],
            'var_smoothing': [1e-9, 1e-8, 1e-7]
        }
    },
    'multinomial_naive_bayes':{
        'model':MultinomialNB(),
        'params':{
            'alpha': [0.1, 0.5, 1.0],
            'fit_prior': [True, False]
        }
    }
}

In [105]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

df3 = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df3

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 10}
3,decision_tree,0.966667,"{'criterion': 'gini', 'max_depth': None, 'min_..."
4,gaussian_naive_bayes,0.953333,"{'priors': None, 'var_smoothing': 1e-09}"
5,multinomial_naive_bayes,0.953333,"{'alpha': 0.1, 'fit_prior': True}"


**From above we can conclude that svm model with its best parameters i.e C=1, kernel=rbf is the best model for classification of iris dataset**
---